In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import flg_model
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools
import glob

fast_mode = False#(fls.env=='local')
clear_data = False

MainProcess


In [2]:
# Prepare datasets
if not fast_mode:
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', fls.result_dir + '/many_full/', skip_download=clear_data)
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr/', skip_download=clear_data)
else:
    if clear_data:
        fls.remove_and_make_dir(fls.result_dir + '/many_full/')
        fls.remove_and_make_dir(fls.result_dir + '/many_abbr/')

In [ ]:
#fls.profiling=True
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count(start=0):
    print('Starting seed ', i)
    r = flg_runner.baseline_runner(fast_mode=fast_mode)
    r.label = 'Test ensemble'
    r.modifier_dict = dict()
    r.modifier_dict['n_ensemble'] = flg_runner.pm(1, lambda r:(r.integers(4,5)).item(), flg_runner.yolo)    
    r.base_model = flg_model.ThreeStepModelLabelBased()
    r.base_model.train_data_selector.datasets = ['tom']
    r.base_model.run_in_parallel = False
    if fast_mode: r.base_model.step1Labels.n_epochs = 1
    r.git_commit_id = git_commit_id
    r.env = fls.env
    r.seed = i
    #r.base_model.train_data_selector.datasets = 'tom'
    #r.base_model.run_in_parallel = False
    if r.use_missing_value:
        base_filename = r.label + '_' + str(r.seed) + 'M_' + git_commit_id[:8]   
    else:
        base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]   
        if len(glob.glob(fls.result_dir + '/many_full/' + r.label + '_' + str(r.seed) + '_' + '*'))>=1:
            print('Skipping')
            continue
    r.run()

    score_str = str(r.cv_score)[2:5]
    print(str(r.cv_score)[1:5])
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_' + score_str +' _f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_' + score_str +' _a.pickle'
    
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode:
        fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
        fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

Starting seed  0
1919 13
30 13
{'seed': 0, 'n_ensemble': 4}
Clearing cupy
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
Processing tomogram tomo_512f98 (1/1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
Processing 800 out of 800 slices (CONCENTRATION=1)
[{'z': 488, 'y': 876, 'x': 792, 'confidence': 0.275146484375, 'i_model': 0}, {'z': 489, 'y': 876, 'x': 791, 'confidence': 0.323486328125, 'i_model': 0}, {'z': 490, 'y': 876, 'x': 790, 'confidence': 0.366943359375, 'i_model': 0}, {'z': 491, 'y': 875, 'x': 789, 'confidence': 0.277587890625, 'i_model': 0}, {'z': 501, 'y': 876, 'x': 782, 'confidence': 0.2602

In [ ]:
%debug